In [1]:
from bs4 import BeautifulSoup
from urllib.request import urlopen
import re
import pandas as pd

In [2]:
# let's grab html from page listing links to individual year results
html_outer = urlopen("https://skyhoundz.com/previous-competition-results/")

# turn it to soup
soup_outer = BeautifulSoup(html_outer,'html.parser')

# view it
# print(soup_outer.prettify())

In [3]:
# store each link that connects to a page of xtreme distance results
result_links = []

for a in soup_outer.find_all('a'):
    if re.search('skyhoundz-classic-results',a['href']):
        result_links.append(a['href'])

result_links

['https://skyhoundz.com/2020-skyhoundz-classic-results/',
 'https://skyhoundz.com/2019-skyhoundz-classic-results/',
 'https://skyhoundz.com/2018-skyhoundz-classic-results/',
 'https://skyhoundz.com/2017-skyhoundz-classic-results/',
 'https://skyhoundz.com/2016-skyhoundz-classic-results/',
 'https://skyhoundz.com/2015-skyhoundz-classic-results/',
 'https://skyhoundz.com/2014-skyhoundz-classic-results/',
 'https://skyhoundz.com/2013-skyhoundz-classic-results/',
 'https://skyhoundz.com/2012-skyhoundz-classic-results/',
 'https://skyhoundz.com/2011-skyhoundz-classic-results/',
 'https://skyhoundz.com/2010-skyhoundz-classic-results/',
 'https://skyhoundz.com/2009-skyhoundz-classic-results/',
 'https://skyhoundz.com/2008-skyhoundz-classic-results/',
 'https://skyhoundz.com/2007-skyhoundz-classic-results/',
 'https://skyhoundz.com/2006-skyhoundz-classic-results/',
 'https://skyhoundz.com/2005-skyhoundz-classic-results/',
 'https://skyhoundz.com/2004-skyhoundz-classic-results/',
 'https://skyh

In [4]:
######### eventually hoping to loop through these links and scrape tables from each page #########



In [5]:
#### let's pull one table from the 2019 results page
html_inner = urlopen(result_links[1])
soup_inner = BeautifulSoup(html_inner,'html.parser')
# print(soup_inner.prettify())

In [6]:
def get_data(html):
    data = []
    with urlopen(html) as fp:
        soup = BeautifulSoup(fp, 'html.parser')
        rows = soup.find_all('tr') #Find all table row tags
        for row in rows: #For each row in that tag
            cols = row.find_all('td') #Find all the columns
            cols = [ele.text.strip() for ele in cols]  #Trim the junk off the cols
            data.append([ele if ele else '' for ele in cols])   #Append while ignoring empty vals
    return data

def parse_data(data):
    titles = []
    tables = {}
    for row in data:
        if len(row) == 1:
            current_title = row[0]
            tables[current_title] = []
            titles.append(current_title)
        elif len(row) != 0:
            tables[current_title].append(row)
    return(titles, tables)


In [8]:
data = get_data(result_links[1])
titles, tables = parse_data(data)
for title in tables:
    print(title)

2019 / World Championship / Chattanooga, Tennessee / 9/22/2019 / Open
2019 / World Championship / Chattanooga, Tennessee / 9/22/2019 / Open (MicroDog)
2019 / World Championship / Chattanooga, Tennessee / 9/22/2019 / Open (Youth)
2019 / World Championship / Chattanooga, Tennessee / 9/22/2019 / Pairs
2019 / World Championship / Chattanooga, Tennessee / 9/22/2019 / Sport
2019 / World Championship / Chattanooga, Tennessee / 9/22/2019 / Sport (MicroDog)
2019 / Last Chance Qualifier / Chattanooga, Tennessee / 9/20/2019 / Open
2019 / Last Chance Qualifier / Chattanooga, Tennessee / 9/20/2019 / Open (MicroDog)
2019 / Last Chance Qualifier / Chattanooga, Tennessee / 9/20/2019 / Open (Youth)
2019 / Last Chance Qualifier / Chattanooga, Tennessee / 9/20/2019 / Pairs
2019 / Last Chance Qualifier / Chattanooga, Tennessee / 9/20/2019 / Sport
2019 / Last Chance Qualifier / Chattanooga, Tennessee / 9/20/2019 / Sport (MicroDog)
2019 / Open Qualifier / Auburn, Washington / 8/17/2019 / Open
2019 / Open Qu

In [9]:
tables[titles[1]][0]

['1',
 'Dean Werts & Glamour',
 '9.5',
 '10.0',
 '10.0',
 '9.5',
 '39.0',
 '15.0',
 '9.0',
 '9.5',
 '9.5',
 '8.5',
 '36.5',
 '90.5',
 '']

In [10]:
titles[0]

'2019 / World Championship / Chattanooga, Tennessee / 9/22/2019 / Open'

In [11]:
titles[0].split(" / ")

['2019', 'World Championship', 'Chattanooga, Tennessee', '9/22/2019', 'Open']

In [12]:
# appends the descriptors stored in title to the lists and creates pd df of that subtable
# subtables added to existing df for j>0
def get_pandas(titles_in, tables_in):
    for j in range(len(tables_in)):
        for i in range(len(tables_in[titles_in[j]])):
            desc = titles_in[j].split(" / ")
            for string in desc:
                tables_in[titles_in[j]][i].append(string)
        if j==0:
            t = pd.DataFrame(tables_in[titles_in[j]], 
                              columns=['Placement','Team','Score','TitleEligibile','Year','Event','Date','Division'])
        else:
            t = t.append(pd.DataFrame(tables_in[titles_in[j]], 
                       columns=['Placement','Team','Score','TitleEligibile',
                                'Year','Event','Date','Division']))
    return t

In [13]:
# this function pulls all Xtreme Distance data from Skyhoundz website

def XtremeDist_WebScrape():
    # let's grab html from page listing links to individual year results
    html_outer = urlopen("https://skyhoundz.com/previous-competition-results/")

    # turn it to soup
    soup_outer = BeautifulSoup(html_outer,'html.parser')
    
    # gather each link containing xtreme distance results
    result_links = []
    for a in soup_outer.find_all('a'):
        if re.search('skyhoundz-classic-results',a['href']):
            result_links.append(a['href'])
    
    # the following loop cycles through each url containing results (1 page per year of results)
    # and creates pandas dataframe of all results combined
    for k in range(len(result_links)):
    
        html_inner = urlopen(result_links[k])
        soup_inner = BeautifulSoup(html_inner,'html.parser')
    
        data = get_data(result_links[k])
        titles, tables = parse_data(data)
    
        if k == 0:
            p = get_pandas(titles, tables)
        else:
            p = p.append(get_pandas(titles, tables))
        
    return p

In [14]:
dat = XtremeDist_WebScrape()
dat

ValueError: 8 columns passed, passed data had 20 columns

In [18]:
#dat.to_csv('C:/Users/Amber/Documents/Projects/DIODE/data/XtremeDistanceResults.csv')